In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
import math

In [ ]:
import numpy as np

In [ ]:
from numpy import arange
from timeit import Timer

Nelements = 10000
Ntimeits = 10000

x = arange(Nelements)
y = range(Nelements)

t_numpy = Timer("x.sum()", "from __main__ import x")
t_list = Timer("sum(y)", "from __main__ import y")
print("numpy: %.3e" % (t_numpy.timeit(Ntimeits)/Ntimeits,))
print("list:  %.3e" % (t_list.timeit(Ntimeits)/Ntimeits,))

# Pick out unique rows

In [ ]:
x = np.array([[1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 0]])

In [ ]:
np.unique(x, axis=0)
np.unique(x)
np.unique(x, axis = 1)

In [ ]:
x.shape

In [ ]:
y = np.array([[1],[2],[3],[4],[5],[6]])

In [ ]:
y.shape

In [ ]:
z = np.dot(x, y)
z.shape

In [ ]:
np.sin(np.pi)

In [ ]:
np.sin(1.5707963267948966)

In [ ]:
np.arcsin(1)

In [ ]:
np.pi

In [ ]:
np.sin(0)
np.sin(2*np.pi)

<span style="font-family: Cosmic Sans MS; font-size:2em; color: #808000">
$$
P ( N ) = \left( \begin{array} { c } { n } \\ { N } \end{array} \right) p ^ { N } ( 1 - p ) ^ { n - N }
$$
    
binomial(n, p, size=None)

In [ ]:
np.random.binomial(2,0.3)

In [ ]:
np.random.binomial(2,0.1, size=300)

In [ ]:
np.random.chisquare()

In [ ]:
math.log(10)

In [ ]:
def Pkernel(x):
    """Kernel function from Biteau & Williams (2015), Eq. (7)"""

    m = (x < 0.) & (x >= 1.)
    x[x < 0.] = np.zeros(np.sum(x < 0.))
    x[x >= 1.] = np.zeros(np.sum(x >= 1.))
    x = np.sqrt(x)

    result = np.log(2.) * np.log(2.)  - np.pi *np.pi / 6. \
	    + 2. * spence(0.5 + 0.5 * x) - (x + x*x*x) / (1. - x*x) \
	    + (np.log(1. + x) - 2. * np.log(2.)) * np.log(1. - x) \
	    + 0.5 * (np.log(1. - x) * np.log(1. - x) - np.log(1. + x) * np.log(1. + x)) \
	    + 0.5 * (1. + x*x*x*x) / (1. - x*x) * (np.log(1. + x) - np.log(1. - x))
    result[x <= 0.] = np.zeros(np.sum(x <= 0.))
    result[x >= 1.] = np.zeros(np.sum(x >= 1.))
    return result

In [ ]:
np.log10(1e4)

In [ ]:
3e4

In [ ]:
np.zeros([3,4], dtype='int')
np.ones([3,4], dtype='int') 
np.linspace(-7, 2, 10, 0) 

In [ ]:
ar2d = np.array([[1, 2, 3], [11, 12, 13], [10, 20, 40]], dtype='uint8', order='C')

In [ ]:
ar2df = np.array([[1, 2, 3], [11, 12, 13], [10, 20, 40]], dtype='uint8', order='F')

In [ ]:
"""
===================================
Shaded & power normalized rendering
===================================

The Mandelbrot set rendering can be improved by using a normalized recount
associated with a power normalized colormap (gamma=0.3). Rendering can be
further enhanced thanks to shading.

The `maxiter` gives the precision of the computation. `maxiter=200` should
take a few seconds on most modern laptops.
"""
import numpy as np


def mandelbrot_set(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon=2.0):
    X = np.linspace(xmin, xmax, xn, dtype=np.float32)
    Y = np.linspace(ymin, ymax, yn, dtype=np.float32)
    C = X + Y[:, None]*1j
    N = np.zeros(C.shape, dtype=int)
    Z = np.zeros(C.shape, np.complex64)
    for n in range(maxiter):
        I = np.less(abs(Z), horizon)
        N[I] = n
        Z[I] = Z[I]**2 + C[I]
    N[N == maxiter-1] = 0
    return Z, N


if __name__ == '__main__':
    import time
    import matplotlib
    from matplotlib import colors
    import matplotlib.pyplot as plt

    xmin, xmax, xn = -2.25, +0.75, 3000/2
    ymin, ymax, yn = -1.25, +1.25, 2500/2
    maxiter = 200
    horizon = 2.0 ** 40
    log_horizon = np.log(np.log(horizon))/np.log(2)
    Z, N = mandelbrot_set(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon)

    # Normalized recount as explained in:
    # https://linas.org/art-gallery/escape/smooth.html
    # https://www.ibm.com/developerworks/community/blogs/jfp/entry/My_Christmas_Gift

    # This line will generate warnings for null values but it is faster to
    # process them afterwards using the nan_to_num
    with np.errstate(invalid='ignore'):
        M = np.nan_to_num(N + 1 -
                          np.log(np.log(abs(Z)))/np.log(2) +
                          log_horizon)

    dpi = 72
    width = 10
    height = 10*yn/xn
    fig = plt.figure(figsize=(width, height), dpi=dpi)
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False, aspect=1)

    # Shaded rendering
    light = colors.LightSource(azdeg=315, altdeg=10)
    M = light.shade(M, cmap=plt.cm.hot, vert_exag=1.5,
                    norm=colors.PowerNorm(0.3), blend_mode='hsv')
    plt.imshow(M, extent=[xmin, xmax, ymin, ymax], interpolation="bicubic")
    ax.set_xticks([])
    ax.set_yticks([])

    # Some advertisement for matplotlib
    year = time.strftime("%Y")
    major, minor, micro = matplotlib.__version__.split('.', 2)
    text = ("The Mandelbrot fractal set\n"
            "Rendered with matplotlib %s.%s, %s - http://matplotlib.org"
            % (major, minor, year))
    ax.text(xmin+.025, ymin+.025, text, color="white", fontsize=12, alpha=0.5)

    plt.show()